In [1]:
from tools import *
import sys

In [2]:
ecpy_path = '../../../ecpy/'
sys.path.append(os.path.abspath(ecpy_path))
import utils

In [3]:
def test(ecofile,infile,outfile,ite=1):
    # ecofile, ecoli model file prefix
    # infile, the model file for gap-filling
    # outfile, the output model file
    eco = load_ecoli_model(ecofile+'.mat',ecofile+'.xml')
    model = pickle.load(open(infile,'rb'))
    
    print('Convert eco to universal model') 
    universal = convert_eco_to_metacyc_kegg(eco,model)
    return universal,model

In [4]:
def print_rxn_of_met(model,met_id):
    met = model.metabolites.get_by_id(met_id)
    for rxn in met.reactions:
        if 'biomass' in rxn.id.lower():continue
        if rxn.metabolites[met]<0: continue
        print(rxn.id,rxn.reaction)

universal,model = test(ecofile_prefix,model_file3,outfile1)

utils.test_met_production(universal,'CPD-15815_c')

print_rxn_of_met(model,'CPD-15815_c')

In [5]:
def some_manual_curations(model):
    try:
        # based on metacyc website
        rxn = model.reactions.get_by_id('NADH-DEHYDROG-A-RXN')
        rxn.lower_bound = 0
        print(rxn.reaction)
    except: pass
    

In [6]:
def block_some_universal_rxns(model):
    rxn = model.reactions.get_by_id('FE3DCITexs')
    rxn.upper_bound = 0
    

In [7]:
def add_missing_tag(rxn):
    rxn_clean = rxn.copy()
    # add _missing tag for those genes
    new_gr = rxn_clean.gene_reaction_rule
    for gene in parse_gr(rxn_clean.gene_reaction_rule):
        new_gr = new_gr.replace(gene,gene+'_missing')
    rxn_clean.gene_reaction_rule = new_gr
    return rxn_clean

In [8]:
def manually_fill_ions_uptake(universal,model):
    rxns_to_add = []
    def _add_rxn(rxn_id):
        rxn = universal.reactions.get_by_id(rxn_id)
        rxn_clean = add_missing_tag(rxn)
        
        rxns_to_add.append(rxn_clean)
        print(rxn_clean.id, rxn_clean.reaction,rxn_clean.gene_reaction_rule)
        print()
    
    # K+ 
    _add_rxn('TRANS-RXN-3')
    
    # CO+2_c
    _add_rxn('TRANS-RXN-141A_copy2')
    
    # CU+2_c
    _add_rxn('RXN0-14_copy2')
    
    # FE+2_c 
    _add_rxn('TRANS-RXN-8_copy2')
    
    # FE+3_c 
    _add_rxn('3.6.3.30-RXN')
    
    # MN+2_c
    _add_rxn('RXN0-16_copy2')
    
    model.add_reactions(rxns_to_add)

In [9]:
def gap_filling_for_lipid(universal,model):
    for met_id in ['CPD-12819_p','CPD-17086_p','KDO2-LIPID-IVA_e']:
        s = utils.test_met_production(model,met_id)
        rxns_to_add = []
        if s.objective_value == 0:
            su = utils.test_met_production(universal,met_id)
            for rxn_id in su.fluxes.index:
                if rxn_id == 'tmp_sink': continue
                if rxn_id == 'ATPASE-RXN_copy1': continue
                if rxn_id.startswith('EX_'):continue
                if su.fluxes[rxn_id] == 0: continue
                try: model.reactions.get_by_id(rxn_id)
                except: 
                    rxn = universal.reactions.get_by_id(rxn_id)
                    rxn = add_missing_tag(rxn)
                    rxns_to_add.append(rxn)
                    print('  ',rxn.id,rxn.reaction)
        if len(rxns_to_add) >0: model.add_reactions(rxns_to_add)
        print(len(rxns_to_add), 'for lipid',met_id)
        model.repair()
       

In [10]:
def run_gapfilling(ecofile,infile,outfile,ite=1):
    # ecofile, ecoli model file prefix
    # infile, the model file for gap-filling
    # outfile, the output model file
    eco = load_ecoli_model(ecofile+'.mat',ecofile+'.xml')
    model = pickle.load(open(infile,'rb'))
    
    some_manual_curations(model)
    
    print('Convert eco to universal model') 
    universal = convert_eco_to_metacyc_kegg(eco,model)
    block_some_universal_rxns(universal)
    print()
    print('Test if there is consistant prediction before and after conversion')
    report_model_status(eco)
    report_model_status(universal)
    
    universal.objective = 'BIOMASS_Ec_iML1515_core_75p37M'
    eco.objective = 'BIOMASS_Ec_iML1515_core_75p37M'
    print(universal.optimize())
    print(eco.optimize())
    print()
    
    
    print('Test if model can produce components in biomass with the medium without YE')
    utils.set_yeast_extraction(model,lb=0,ub=0)
    check_if_model_can_produce_mets_in_biomass_one_by_one(model)
    print()
    
    print('Do gap-filling one by one')
    manually_fill_ions_uptake(universal,model)
    gap_filling_for_lipid(universal,model)
    for i in range(ite):
        for met in  model.reactions.Biomass_v1.reactants:
            if met.id == 'ions_c': continue
            if met.id == 'lipid_c': continue
            do_gap_filling(met,model,universal)
        
    print('Test if model can produce components in biomass with the medium without YE, 1 glucose')
    model.reactions.Exchange_Glucopyranose.lower_bound = 0
    model.reactions.Exchange_Glucopyranose.upper_bound = 1
    check_if_model_can_produce_mets_in_biomass_one_by_one(model)
    print()
    
    print('Test biomass production with/wihout YE')
    print(utils.test_biomass_production(model))
    print()
    
    print('Update missing genes')
    model = update_missing_genes(model,1e-3,report=True,coverage=45)
    print()
    
    print('Save model')
    save_pickle_model(model,outfile)

In [11]:
ecofile_prefix ='../../../ComplementaryData/iML1515'
model_file1 = '../Results/halo_metacycmodel_100_curated_compart_with_eco_medium_biomass.pkl'
outfile1    = '../Results/halo_metacycmodel_100_curated_compart_with_eco_medium_biomass_gapfilled.pkl'

model_file2 = '../Results/halo_metacyc_kegg_curated_compart_with_eco_medium_biomass.pkl'
outfile2    = '../Results/halo_metacyc_kegg_curated_compart_with_eco_medium_biomass_gapfilled.pkl'

model_file3 = '../Results/halo_metacycmodel_100_curated_compart_with_eco_without_missing_medium_biomass.pkl'
outfile3    = '../Results/halo_metacycmodel_100_curated_compart_with_eco_without_missing_medium_biomass_gapfilled.pkl'

model_file4 = '../Results/halo_metacyc_kegg_curated_compart_with_eco_without_missing_medium_biomass.pkl'
outfile4    = '../Results/halo_metacyc_kegg_curated_compart_with_eco_without_missing_medium_biomass_gapfilled.pkl'

In [12]:
run_gapfilling(ecofile_prefix,model_file1,outfile1)
model = pickle.load(open(outfile1,'rb'))
for met in model.metabolites:
    try: met.charge = int(met.charge)
    except: met.charge=None
utils.save_model_into_formats(model,'../../../ModelFiles/','Halo_GEM_v1')

Using license file /Users/gangl/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpr7izw7yn.lp
Reading time = 0.02 seconds
: 2399 rows, 4618 columns, 21886 nonzeros
NADH_c + PROTON_c + Ubiquinones_c --> NAD_c + Ubiquinols_c
Convert eco to universal model
Metacyc 16868
KEGG 37608
Metacyc 14768
KEGG 10278

Test if there is consistant prediction before and after conversion
Number of reactions: 2712
Number of metabolits: 1877
Number of compartments: 3 {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

Number of reactions: 2712
Number of metabolits: 1872
Number of compartments: 3 {'c': '', 'e': '', 'p': ''}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

<Solution 0.877 at 0x132bb93278>
<Solution 0.877 at 0x132bb932e8>

Test if model can produce components 

Ignoring reaction 'TRANS-RXN-3' since it already exists.


   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 0.0


Do gap-filling one by one
TRANS-RXN-3 K+_p + PROTON_p --> K+_c + PROTON_c ( b1363_missing and b1291_missing and b3290_missing ) or ( b1291_missing and b3849_missing and b3290_missing ) or b1250_missing or b3747_missing

TRANS-RXN-141A_copy2 CO+2_p --> CO+2_c b3040_missing or b3816_missing

RXN0-14_copy2 CU+2_p --> CU+2_c b3040_missing

TRANS-RXN-8_copy2 FE+2_p --> FE+2_c b3040_missing

3.6.3.30-RXN ATP_c + CPD-15815_c + FE+3_p --> ADP_c + CPD-16459_c + FE+3_c + PROTON_c 

RXN0-16_copy2 MN+2_p --> MN+2_c b3040_missing

<Solution 0.000 at 0x1328499860>
<Solution 1.143 at 0x1328499898>
   R01068 C00354_c <=> DIHYDROXY-ACETONE-PHOSPHATE_c + GAP_c
   RXN0-313 FRUCTOSE-6P_c <=> DIHYDROXYACETONE_c + GAP_c
   HOMOSERKIN-RXN ATP_c + HOMO-SER_c --> ADP_c + O-PHOSPHO-L-HOMOSERINE_c + PROTON_c
   R00943 ATP_c + CARBOXYL-GROUP_c + THF_c --> 10-FORMYL-THF_c + ADP_c + CPD-16459_c
   R01775 HOMO-SER_c + NADP_c <=> L-ASPARTATE-SE

Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.


After gap-filling: 49.89973112719744
Added reactions:
   DIHYDRONEOPTERIN-MONO-P-DEPHOS-RXN CPD-15815_c + DIHYDRONEOPTERIN-P_c --> CPD-11770_c + CPD-16459_c
   H2NEOPTERINP3PYROPHOSPHOHYDRO-RXN CPD-15815_c + DIHYDRONEOPTERIN-P3_c --> DIHYDRONEOPTERIN-P_c + PPI_c + PROTON_c
   DM_GLYCOLALDEHYDE_c GLYCOLALDEHYDE_c --> 
   DM_CPD-15815_e CPD-15815_e --> 



2fe2s_c
Before gap-filling: 116.095
No need for gap-filling



2-OCTAPRENYL-6-HYDROXYPHENOL_c
Before gap-filling: 34.525877551020415
No need for gap-filling



CPD-7_c
Before gap-filling: 52.770454545454584
No need for gap-filling



S-ADENOSYLMETHIONINE_c
Before gap-filling: 59.21755952380947
No need for gap-filling



BIOTIN_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpm_8kvo30.lp
Reading time = 0.02 seconds
: 2402 rows, 4694 columns, 22152 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp2k_ponon.lp
Reading time = 0.02 seconds


...

After gap-filling: 48.87745535714316
Added reactions:
   PANTOTHENATE-KIN-RXN ATP_c + PANTOTHENATE_c --> 4-P-PANTOTHENATE_c + ADP_c + PROTON_c



FAD_c
Before gap-filling: 41.25954681487822
No need for gap-filling



CPD-15815_c
Before gap-filling: 1000.0
No need for gap-filling



METHYLENE-THF_c
Before gap-filling: 50.15908523908546
No need for gap-filling



CPD0-2278_p
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpjg_wc_tp.lp
Reading time = 0.02 seconds
: 2402 rows, 4706 columns, 22172 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpiu4sbbyu.lp
Reading time = 0.02 seconds
: 1872 rows, 5424 columns, 21150 nonzeros


Ignoring reaction 'TETHYDPICSUCC-RXN' since it already exists.
Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.


After gap-filling: 41.66916256157636
Added reactions:
   DM_CPD-15815_c CPD-15815_c --> 
   DM_D-ALANINE_p D-ALANINE_p --> 



NAD_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp2fhg8w42.lp
Reading time = 0.02 seconds
: 2402 rows, 4710 columns, 22176 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp0fe6xnej.lp
Reading time = 0.02 seconds
: 1872 rows, 5424 columns, 21150 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.


After gap-filling: 61.118480050665
Added reactions:
   NAD-SYNTH-NH3-RXN AMMONIA_c + ATP_c + DEAMIDO-NAD_c --> AMP_c + NAD_c + PPI_c + PROTON_c
   DM_CPD-15815_p CPD-15815_p --> 



NADP_c
Before gap-filling: 58.45310720775288
No need for gap-filling



PROTOHEME_c
Before gap-filling: 49.402127659574475
No need for gap-filling



PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 455.96685714285724
No need for gap-filling



RIBOFLAVIN_c
Before gap-filling: 76.5505882352941
No need for gap-filling



SIROHEME_c
Before gap-filling: 18.949619047619034
No need for gap-filling



SUC-COA_c
Before gap-filling: 46.70680751173688
No need for gap-filling



THF_c
Before gap-filling: 51.27846971307125
No need for gap-filling



THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 65.56119565217392
No need for gap-filling



CPD-9649_c
Before gap-filling: 29.143290267011196
No need for gap-filling



ATP_c
Before gap-filling: 93.69522330097095
No need for gap-filling



Test if model can produce components in bi

In [13]:
run_gapfilling(ecofile_prefix,model_file2,outfile2)
model = pickle.load(open(outfile2,'rb'))
for met in model.metabolites:
    try: met.charge = int(met.charge)
    except: met.charge=None
utils.save_model_into_formats(model,'../../../ModelFiles/','Halo_GEM_v2')

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp4crf9gow.lp
Reading time = 0.02 seconds
: 3310 rows, 6314 columns, 29474 nonzeros
NADH_c + PROTON_c + Ubiquinones_c --> NAD_c + Ubiquinols_c
Convert eco to universal model
Metacyc 16868
KEGG 37608
Metacyc 14768
KEGG 10278

Test if there is consistant prediction before and after conversion
Number of reactions: 2712
Number of metabolits: 1877
Number of compartments: 3 {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

Number of reactions: 2712
Number of metabolits: 1872
Number of compartments: 3 {'c': '', 'e': '', 'p': ''}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

<Solution 0.877 at 0x132ef6cb00>
<Solution 0.877 at 0x132ef6cbe0>

Test if model can produce components in biomass with the medium without YE
Biomass_v1
   protein_c protein 0.0
   dna_c dna 0.0

Ignoring reaction 'TRANS-RXN-3' since it already exists.


   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 0.0


Do gap-filling one by one
TRANS-RXN-3 K+_p + PROTON_p --> K+_c + PROTON_c ( b1363_missing and b1291_missing and b3290_missing ) or ( b1291_missing and b3849_missing and b3290_missing ) or b1250_missing or b3747_missing

TRANS-RXN-141A_copy2 CO+2_p --> CO+2_c b3040_missing or b3816_missing

RXN0-14_copy2 CU+2_p --> CU+2_c b3040_missing

TRANS-RXN-8_copy2 FE+2_p --> FE+2_c b3040_missing

3.6.3.30-RXN ATP_c + CPD-15815_c + FE+3_p --> ADP_c + CPD-16459_c + FE+3_c + PROTON_c 

RXN0-16_copy2 MN+2_p --> MN+2_c b3040_missing

<Solution 0.000 at 0x132a4f4cc0>
<Solution 1.143 at 0x13284994e0>
   R01068 C00354_c <=> DIHYDROXY-ACETONE-PHOSPHATE_c + GAP_c
   RXN0-313 FRUCTOSE-6P_c <=> DIHYDROXYACETONE_c + GAP_c
   R00943 ATP_c + CARBOXYL-GROUP_c + THF_c --> 10-FORMYL-THF_c + ADP_c + CPD-16459_c
   R01775 HOMO-SER_c + NADP_c <=> L-ASPARTATE-SEMIALDEHYDE_c + NADPH_c + PROTON_c
   RXN-14452 PROTON_e <=> PROTON_p
   TRANS-RXN0-47

Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_NITRATE_c' since it already exists.


After gap-filling: 61.643835616438196
Added reactions:
   DIHYDRONEOPTERIN-MONO-P-DEPHOS-RXN CPD-15815_c + DIHYDRONEOPTERIN-P_c --> CPD-11770_c + CPD-16459_c
   H2NEOPTERINP3PYROPHOSPHOHYDRO-RXN CPD-15815_c + DIHYDRONEOPTERIN-P3_c --> DIHYDRONEOPTERIN-P_c + PPI_c + PROTON_c
   DM_CPD0-2189_c CPD0-2189_c --> 



2fe2s_c
Before gap-filling: 177.60653465346536
No need for gap-filling



2-OCTAPRENYL-6-HYDROXYPHENOL_c
Before gap-filling: 54.951467889908265
No need for gap-filling



CPD-7_c
Before gap-filling: 82.66479262672806
No need for gap-filling



S-ADENOSYLMETHIONINE_c
Before gap-filling: 80.2535048231513
No need for gap-filling



BIOTIN_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpubce8dk_.lp
Reading time = 0.02 seconds
: 3313 rows, 6376 columns, 29682 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpk0zsgzcj.lp
Reading time = 0.02 seconds
: 1872 rows, 5424 columns, 21150 no

...

After gap-filling: 111.95719626168234
Added reactions:
   DM_DIHYDROXYPENTANEDIONE_c DIHYDROXYPENTANEDIONE_c --> 
   DM_CPD0-2167_c CPD0-2167_c --> 
   DM_ALCOHOL-GROUP_c ALCOHOL-GROUP_c --> 



CO-A_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpc9a8hc1j.lp
Reading time = 0.03 seconds
: 3313 rows, 6382 columns, 29688 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpzy7igtxg.lp
Reading time = 0.02 seconds
: 1872 rows, 5424 columns, 21150 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_NITRATE_c' since it already exists.


After gap-filling: 57.69230769230767
Added reactions:
   PANTOTHENATE-KIN-RXN ATP_c + PANTOTHENATE_c --> 4-P-PANTOTHENATE_c + ADP_c + PROTON_c



FAD_c
Before gap-filling: 44.55445544554467
No need for gap-filling



CPD-15815_c
Before gap-filling: 1000.0
No need for gap-filling



METHYLENE-THF_c
Before gap-filling: 60.810810810810835
No need for gap-filling



CPD0-2278_p
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpo5kfjlbj.lp
Reading time = 0.03 seconds
: 3313 rows, 6384 columns, 29698 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpbpgmj3e9.lp
Reading time = 0.02 seconds
: 1872 rows, 5424 columns, 21150 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_NITRATE_c' since it already exists.


After gap-filling: 29.22077922077922
Added reactions:
   TRANS-RXN-62A D-ALANINE_p + PROTON_p --> D-ALANINE_c + PROTON_c



NAD_c
Before gap-filling: 57.69230769230785
No need for gap-filling



NADP_c
Before gap-filling: 54.878048780487845
No need for gap-filling



PROTOHEME_c
Before gap-filling: 44.55445544554454
No need for gap-filling



PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 187.50000000000003
No need for gap-filling



RIBOFLAVIN_c
Before gap-filling: 86.36276816608952
No need for gap-filling



SIROHEME_c
Before gap-filling: 41.284403669724774
No need for gap-filling



SUC-COA_c
Before gap-filling: 52.94117647058823
No need for gap-filling



THF_c
Before gap-filling: 64.28571428571422
No need for gap-filling



THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 78.46306451612908
No need for gap-filling



CPD-9649_c
Before gap-filling: 47.34948616600789
No need for gap-filling



ATP_c
Before gap-filling: 86.20689655172396
No need for gap-filling



Test if model can produce com

In [14]:
run_gapfilling(ecofile_prefix,model_file3,outfile3,ite=2)

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpz3sh4bjj.lp
Reading time = 0.02 seconds
: 2337 rows, 4472 columns, 21114 nonzeros
NADH_c + PROTON_c + Ubiquinones_c --> NAD_c + Ubiquinols_c
Convert eco to universal model
Metacyc 16868
KEGG 37608
Metacyc 14768
KEGG 10278

Test if there is consistant prediction before and after conversion
Number of reactions: 2712
Number of metabolits: 1877
Number of compartments: 3 {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

Number of reactions: 2712
Number of metabolits: 1872
Number of compartments: 3 {'c': '', 'e': '', 'p': ''}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

<Solution 0.877 at 0x133465a1d0>
<Solution 0.877 at 0x13362d1e10>

Test if model can produce components in biomass with the medium without YE
Biomass_v1
   protein_c protein 0.0
   dna_c dna 0.0

Ignoring reaction 'TRANS-RXN-3' since it already exists.


   CPD-12819_p Phosphatidylethanolamine (dihexadecanoyl, n-C16:0) 0.0
   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 0.0


Do gap-filling one by one
TRANS-RXN-3 K+_p + PROTON_p --> K+_c + PROTON_c ( b1363_missing and b1291_missing and b3290_missing ) or ( b1291_missing and b3849_missing and b3290_missing ) or b1250_missing or b3747_missing

TRANS-RXN-141A_copy2 CO+2_p --> CO+2_c b3040_missing or b3816_missing

RXN0-14_copy2 CU+2_p --> CU+2_c b3040_missing

TRANS-RXN-8_copy2 FE+2_p --> FE+2_c b3040_missing

3.6.3.30-RXN ATP_c + CPD-15815_c + FE+3_p --> ADP_c + CPD-16459_c + FE+3_c + PROTON_c 

RXN0-16_copy2 MN+2_p --> MN+2_c b3040_missing

<Solution 0.000 at 0x1326816da0>
<Solution 1.143 at 0x1326816748>
   R01068 C00354_c <=> DIHYDROXY-ACETONE-PHOSPHATE_c + GAP_c
   PYRUVFORMLY-RXN CO-A_c + PYRUVATE_c --> ACETYL-COA_c + CARBOXYL-GROUP_c
   2.7.1.121-RXN DIHYDROXYACETONE_c + PHOSPHO-ENOL-PYRUVATE_c --> DIHYDROXY-ACETONE-PHOSPHATE_c + PYRUVATE_c
   RXN0-313 FRUCTOSE-

Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.


After gap-filling: 49.212687404385576
Added reactions:
   DIHYDRONEOPTERIN-MONO-P-DEPHOS-RXN CPD-15815_c + DIHYDRONEOPTERIN-P_c --> CPD-11770_c + CPD-16459_c
   H2NEOPTERINP3PYROPHOSPHOHYDRO-RXN CPD-15815_c + DIHYDRONEOPTERIN-P3_c --> DIHYDRONEOPTERIN-P_c + PPI_c + PROTON_c
   DM_CPD0-2189_c CPD0-2189_c --> 
   DM_CPD-15815_p CPD-15815_p --> 



2fe2s_c
Before gap-filling: 107.16461538461542
No need for gap-filling



2-OCTAPRENYL-6-HYDROXYPHENOL_c
Before gap-filling: 34.52587755102041
No need for gap-filling



CPD-7_c
Before gap-filling: 52.770454545454555
No need for gap-filling



S-ADENOSYLMETHIONINE_c
Before gap-filling: 59.217559523809584
No need for gap-filling



BIOTIN_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp4vp9bcjk.lp
Reading time = 0.02 seconds
: 2342 rows, 4562 columns, 21454 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp9y28nvd_.lp
Reading time = 0.02 second

...

After gap-filling: 41.61790897908979
Added reactions:
   TRANS-RXN-62A D-ALANINE_p + PROTON_p --> D-ALANINE_c + PROTON_c



NAD_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpy56mzke1.lp
Reading time = 0.02 seconds
: 2342 rows, 4578 columns, 21484 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpm99_pxzs.lp
Reading time = 0.02 seconds
: 1872 rows, 5424 columns, 21150 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.


After gap-filling: 61.11848005066548
Added reactions:
   NAD-SYNTH-NH3-RXN AMMONIA_c + ATP_c + DEAMIDO-NAD_c --> AMP_c + NAD_c + PPI_c + PROTON_c
   DM_CPD-15815_e CPD-15815_e --> 



NADP_c
Before gap-filling: 58.45310720775285
No need for gap-filling



PROTOHEME_c
Before gap-filling: 49.402127659574475
No need for gap-filling



PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 430.136
No need for gap-filling



RIBOFLAVIN_c
Before gap-filling: 76.55058823529403
No need for gap-filling



SIROHEME_c
Before gap-filling: 18.949619047619045
No need for gap-filling



SUC-COA_c
Before gap-filling: 46.706807511737
No need for gap-filling



THF_c
Before gap-filling: 50.55321110529087
No need for gap-filling



THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp4hfax0m8.lp
Reading time = 0.02 seconds
: 2342 rows, 4582 columns, 21500 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_

Ignoring reaction 'SULFITE-REDUCT-RXN' since it already exists.
Ignoring reaction 'ADENYLYLSULFKIN-RXN' since it already exists.
Ignoring reaction 'TRANS-RXN0-460_copy2' since it already exists.
Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_CO3_c' since it already exists.


After gap-filling: 65.56119565217391
Added reactions:
   THZPSN3 ATP_c + CPD-12279_c + DEOXYXYLULOSE-5P_c + NADPH_c + PROTON_c + iscssh_c --> AMP_c + CARBON-DIOXIDE_c + 2.0 CPD-15815_c + NADP_c + PPI_c + THZ-P_c + iscs_c



CPD-9649_c
Before gap-filling: 29.143290267011196
No need for gap-filling



ATP_c
Before gap-filling: 93.31393025447693
No need for gap-filling



protein_c
Before gap-filling: 112.33954702593466
No need for gap-filling



dna_c
Before gap-filling: 965.7593530552489
No need for gap-filling



rna_c
Before gap-filling: 154.6716987971585
No need for gap-filling



lipids_c
Before gap-filling: 412.01569614996043
No need for gap-filling



10-FORMYL-THF_c
Before gap-filling: 49.212687404385406
No need for gap-filling



2fe2s_c
Before gap-filling: 107.16461538461539
No need for gap-filling



2-OCTAPRENYL-6-HYDROXYPHENOL_c
Before gap-filling: 34.525877551020415
No need for gap-filling



CPD-7_c
Before gap-filling: 52.770454545454534
No need for gap-filling



S-ADENOS

In [15]:
run_gapfilling(ecofile_prefix,model_file4,outfile4,ite=2)

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmppnk7utdo.lp
Reading time = 0.02 seconds
: 3252 rows, 6170 columns, 28714 nonzeros
NADH_c + PROTON_c + Ubiquinones_c --> NAD_c + Ubiquinols_c
Convert eco to universal model
Metacyc 16868
KEGG 37608
Metacyc 14768
KEGG 10278

Test if there is consistant prediction before and after conversion
Number of reactions: 2712
Number of metabolits: 1877
Number of compartments: 3 {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

Number of reactions: 2712
Number of metabolits: 1872
Number of compartments: 3 {'c': '', 'e': '', 'p': ''}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

<Solution 0.877 at 0x132ef0e208>
<Solution 0.877 at 0x132ef0e358>

Test if model can produce components in biomass with the medium without YE
Biomass_v1
   protein_c protein 0.0
   dna_c dna 0.0

Ignoring reaction 'TRANS-RXN-3' since it already exists.


   CPD-12819_p Phosphatidylethanolamine (dihexadecanoyl, n-C16:0) 0.0
   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 0.0


Do gap-filling one by one
TRANS-RXN-3 K+_p + PROTON_p --> K+_c + PROTON_c ( b1363_missing and b1291_missing and b3290_missing ) or ( b1291_missing and b3849_missing and b3290_missing ) or b1250_missing or b3747_missing

TRANS-RXN-141A_copy2 CO+2_p --> CO+2_c b3040_missing or b3816_missing

RXN0-14_copy2 CU+2_p --> CU+2_c b3040_missing

TRANS-RXN-8_copy2 FE+2_p --> FE+2_c b3040_missing

3.6.3.30-RXN ATP_c + CPD-15815_c + FE+3_p --> ADP_c + CPD-16459_c + FE+3_c + PROTON_c 

RXN0-16_copy2 MN+2_p --> MN+2_c b3040_missing

<Solution 0.000 at 0x132bdbe630>
<Solution 1.143 at 0x132bdbe400>
   R01068 C00354_c <=> DIHYDROXY-ACETONE-PHOSPHATE_c + GAP_c
   PYRUVFORMLY-RXN CO-A_c + PYRUVATE_c --> ACETYL-COA_c + CARBOXYL-GROUP_c
   2.7.1.121-RXN DIHYDROXYACETONE_c + PHOSPHO-ENOL-PYRUVATE_c --> DIHYDROXY-ACETONE-PHOSPHATE_c + PYRUVATE_c
   RXN0-313 FRUCTOSE-

Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_NITRATE_c' since it already exists.


After gap-filling: 61.64383561643834
Added reactions:
   DIHYDRONEOPTERIN-MONO-P-DEPHOS-RXN CPD-15815_c + DIHYDRONEOPTERIN-P_c --> CPD-11770_c + CPD-16459_c
   H2NEOPTERINP3PYROPHOSPHOHYDRO-RXN CPD-15815_c + DIHYDRONEOPTERIN-P3_c --> DIHYDRONEOPTERIN-P_c + PPI_c + PROTON_c
   DM_CPD0-2189_c CPD0-2189_c --> 



2fe2s_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp_curvnwb.lp
Reading time = 0.03 seconds
: 3257 rows, 6242 columns, 28974 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpn1zyy6p4.lp
Reading time = 0.02 seconds
: 1872 rows, 5424 columns, 21150 nonzeros
Gap-filling failed for 2fe2s_c



2-OCTAPRENYL-6-HYDROXYPHENOL_c
Before gap-filling: 54.95146788990825
No need for gap-filling



CPD-7_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp7yfmk5pa.lp
Reading time = 0.03 seconds
: 3257 rows, 6242 columns, 28974 nonzeros

Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_NITRATE_c' since it already exists.


After gap-filling: 29.22077922077921
Added reactions:
   TRANS-RXN-62A D-ALANINE_p + PROTON_p --> D-ALANINE_c + PROTON_c



NAD_c
Before gap-filling: 57.69230769230734
No need for gap-filling



NADP_c
Before gap-filling: 54.878048780487916
No need for gap-filling



PROTOHEME_c
Before gap-filling: 44.554455445544534
No need for gap-filling



PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 187.5000000000001
No need for gap-filling



RIBOFLAVIN_c
Before gap-filling: 86.36276816608996
No need for gap-filling



SIROHEME_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpe99m3gm4.lp
Reading time = 0.03 seconds
: 3257 rows, 6244 columns, 28982 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpzdedvpee.lp
Reading time = 0.02 seconds
: 1872 rows, 5424 columns, 21150 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_NITRATE_c' since it already exists.


After gap-filling: 41.28440366972477
Added reactions:
   DM_DIHYDROXYPENTANEDIONE_c DIHYDROXYPENTANEDIONE_c --> 



SUC-COA_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpse_mj_s0.lp
Reading time = 0.02 seconds
: 3257 rows, 6246 columns, 28984 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpklbbxsys.lp
Reading time = 0.02 seconds
: 1872 rows, 5424 columns, 21150 nonzeros
Gap-filling failed for SUC-COA_c



THF_c
Before gap-filling: 64.28571428571422
No need for gap-filling



THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpov655ny0.lp
Reading time = 0.03 seconds
: 3257 rows, 6246 columns, 28984 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpa69eyfry.lp
Reading time = 0.02 seconds
: 1872 rows, 5424 columns, 21150 nonzeros
Gap-filling failed for THIAMINE-PY

   CTP_c CTP C9H12N3O14P3 111.77777777777777
   GTP_c GTP C10H12N5O14P3 88.23529411764726
   UTP_c UTP C9H11N2O15P3 111.77777777777779

ions_pool
   CO+2_c Co2+ 1000.0
   CU+2_c Copper 1000.0
   FE+2_c Fe2+ mitochondria 1000.0
   FE+3_c Iron (Fe3+) 1000.0
   K+_c Potassium 1000.0
   MG+2_c Magnesium 1000.0
   MN+2_c Manganese 1000.0
   CPD-3_c Molybdate 1000.0
   NI+2_c Nickel 1000.0
   HSO4_c Sulfate 1000.0
   ZN+2_c Zinc 1000.0
   CA+2_c Calcium 1000.0
   CL-_c Chloride 1000.0
   AMMONIA_c Ammonium 1000.0

lipids_synthesis
   KDO2-LIPID-IVA_e  11.976190476190474
   CPD-12819_p Phosphatidylethanolamine (dihexadecanoyl, n-C16:0) 27.189189189189204
   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 27.18918918918929


Test biomass production with/wihout YE
(<Solution 0.000 at 0x132ad36b70>, <Solution 0.000 at 0x132ad36748>)

Update missing genes
Number of reactions: 3122
Number of metabolits: 3257
Number of compartments: 3 {'c': '', 'p': '', 'e': ''}
Number of genes: 14